<a href="https://colab.research.google.com/github/hjjimmykim/bridge/blob/master/bridge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contract bridge

Rules: https://en.wikipedia.org/wiki/Contract_bridge#Gameplay

2 teams with 2 players each.

Bidding
 - If everyone passes w/o bid, round is "passed out" and not played.
 - Should we include double and redouble?
 - State representation: One-hot vectors of bids made so far? Should also include which player went first (i.e. dealer)? How do you deal with passes? Double and redouble?

# Libraries

In [0]:
# Usual
import numpy as np
import matplotlib.pyplot as plt

from itertools import cycle # Allows for cyclic iteration. Beware infinite loops!

from IPython.display import clear_output # Clears output from screen

# Setup

In [0]:
# Playing cards
suits = ['♣','◆','♥','♠']              # club < diamond < heart < spade
values = np.arange(2,15)                   # 2 < ... < 10 < 11 (jack) < 12 (queen) < 13 (king) < 14 (ace)
values_str = [str(x) for x in range(2,11)] + ['Jack', 'Queen', 'King', 'Ace']
# Index representation = [2♣,...,A♣,2◆,...,A◆,2♥,...,A♥,2♠,...,A♠] (0-51)

# Bidding
trump_suits = ['C','D','H','S','NT'] # NT = no trump suit

bid_actions_str = ['PASS']
for i in range(1,8):
  for ts in trump_suits:
    bid_actions_str.append(str(i)+ts)
# Index representation = [PASS, 1C, 1D, 1H, 1S, 1NT, ..., 7C, 7D, 7H, 7S, 7NT] (0-35)
    
# Playing

# Classes

In [0]:
# Player
class Player:
  hand = None    # List of cards owned by the player
  team = None    # 0 or 1
  role = None    # 'declarer' or 'dummy' or 'defender'
  
  def __init__(self, team):
    self.team = team
    
  def display_hand(self):
    hand_inds = np.nonzero(self.hand)[0]
    hand_suitvalue = [''.join(ind2suitvalue(x)) for x in hand_inds]
    
    print('Hand: ' + ','.join(hand_suitvalue))

# Functions


In [0]:
# ------------------------------------------------------------------------------
# Conversion functions

# Convert from card index (0-51) to 'valuesuit' or [suit, value]
def ind2suitvalue(ind, string = True):
  assert 0 <= ind <= 51
  
  suit_ind = ind // 13
  
  if string:
    str_suitvalue = str(ind % 13 + 2) + suits[suit_ind]
    
    if suit_ind in [1,2]:
      str_suitvalue = '\x1b[31m' + str_suitvalue + '\x1b[0m'
      
    return str_suitvalue
  else:
    return [suits[suit_ind], ind % 13 + 2]
  
# Convert from [suit, value] to card index (0-51)
def suitvalue2ind(suit, value):
  assert suit in suits
  assert value in values
  
  return suits.index(suit) * 13 + (value - 2)

# Convert from value (2-14) to string ('2'-'Ace')
def value2str(value):
  assert 2 <= value <= 14
  
  return values_str[value - 2]

# Convert from bid index (0-35) to string ('PASS, 1C, 1D, 1H, 1S, 1NT, ..., 7NT')
def bid_ind2str(ind):
  assert 0 <= ind <= 35
  
  if ind == 0:
    return 'PASS'
  else:
    num_tricks = str((ind-1) // 5 + 1)
    trump_suit = str(trump_suits[(ind-1) % 5])
    
    return num_tricks + trump_suit

# ------------------------------------------------------------------------------
# Setup functions

def create_hands(player_list):
  # Number of players
  numPlayers = len(player_list)
  
  # Create and shuffle deck
  deck = np.arange(52)
  np.random.shuffle(deck)
  
  # Ensure that the cards can be distributed evenly
  handsize = int(len(deck)/numPlayers)
  assert len(deck)/numPlayers == handsize
  
  for i in range(numPlayers):
    # Player i's portion
    deck_i = deck[i*handsize : (i+1)*handsize]
    
    # Convert to one-hot vector
    hand = np.zeros(len(deck),dtype=int)
    hand[deck_i] = 1
    
    player_list[i].hand = hand

# ------------------------------------------------------------------------------
# Message functions

def bidding_message(i, team, last_bid, declarer, pass_count):
  player_name = ['N','E','W','S'] # Convention
  player_name = [0,1,2,3]
  
  declarer_str = ''
  if declarer != None:
    declarer_str = ' (Team ' + str(player_list[declarer].team) + ', Player ' + str(declarer) + ')'
  
  msg = '---------------' + '\n' + \
        ' Bidding stage ' + '\n' + \
        '---------------' + '\n' + \
        'Current player:\t' + str(player_name[i]) + ' (Team ' + str(team) + ')' + '\n' + \
        'Last bid:\t' + str(last_bid) + declarer_str + '\n' + \
        'Pass count:\t' + str(pass_count) + '\n'
  
  return msg
        
def playing_message(i, team, contract_team, contract):
  player_name = ['N','E','W','S'] # Convention
  player_name = [0,1,2,3]
  
  msg = '---------------' + '\n' + \
        ' Playing stage ' + '\n' + \
        '---------------' + '\n' + \
        'Current player:\t' + str(player_name[i]) + ' (Team ' + str(team) + ')' + '\n' + \
        'Contract:\t' + str(contract) + ' by Team ' + str(contract_team) + '\n'
  
  return msg

# Game

In [49]:
# Setup ------------------------------------------------------------------------

# Create players
player_list = []
for i in range(4):
  player_list.append(Player(i % 2))  # team #'s are [0,1,0,1]

# Create hands
create_hands(player_list)

# Bidding ----------------------------------------------------------------------
input('Press ENTER to begin the bidding stage');

declarer = None # Declarer id
last_bid = None # Last bid
pass_count = 0  # Bidding stops when 3 consecutive passes occur

for i in cycle(range(len(player_list))): # Cycle through players
  player = player_list[i]

  valid_input = False  # Keep asking for input until valid input is received
  show_hand = False    # Don't display immediately in case others are watching
  
  while not valid_input:
    
    # Display instruction
    clear_output()
    print(bidding_message(i, player.team, last_bid, declarer, pass_count))
    
    if show_hand:
      player.display_hand()
      input_msg = 'Select an action (PASS, 1C, 1D, 1H, 1S, 1NT, ..., 7NT): '
    else:
      input_msg = 'Select an action (PASS, 1C, 1D, 1H, 1S, 1NT, ..., 7NT)\n' + \
                  'Or type SHOW to see your hand: '
      
    # Receive user input
    print(input_msg)
    action = input()
    
    if action == 'SHOW':            # Show hand in the next iteration
      show_hand = True
    elif action == 'PASS':          # Pass
      pass_count += 1
      valid_input = True
    elif action in bid_actions_str: # Bid
      
      # Either first bid or must be higher than the last bid
      if last_bid == None or bid_actions_str.index(action) > bid_actions_str.index(last_bid):
        # Change declarer
        cond1 = declarer == None or player_list[declarer].team != player.team # if first bid or different team
        cond2 = last_bid and last_bid[1:] != action[1:]               # or different trump suit
        if cond1 or cond2:
          declarer = i

        # Set bid and reset pass count
        last_bid = action
        pass_count = 0
        
        valid_input = True
      
  # Break if no bids made or 3 consecutive passes after a bid
  if pass_count > 3 or (last_bid != None and pass_count == 3):
    break
    
if last_bid == None:
  raise Exception('No bids were made. Game ends.')
  
player_list[declarer].role = 'declarer'
player_list[(declarer + 2) % 4].role = 'dummy'
player_list
  
input('Press ENTER to begin the playing stage');

# Playing ----------------------------------------------------------------------
player_id = declarer # Declarer goes first
declarer_team = player_list[declarer].team

clear_output()
print(playing_message(player_id, player_list[player_id].team, declarer_team, last_bid))

# Scoring

---------------
 Playing stage 
---------------
Current player:	0 (Team 0)
Contract:	3NT by Team 0

